The starting point for this project is the `Polygon` class and the `Polygons` PolygonsIterator type we created in the previous project.

The code for these classes along with the unit tests for the `Polygon` class are below if you want to use those as your starting point. But use whatever you came up with in the last project.

The starting point for this project is the `Polygon` class and the `Polygons` PolygonsIterator type we created in the previous project.

The code for these classes along with the unit tests for the `Polygon` class are below if you want to use those as your starting point. But use whatever you came up with in the last project.

We have two goals:

##### Goal 1

Refactor the `Polygon` class so that all the calculated properties are lazy properties, i.e. they should still be calculated properties, but they should not have to get recalculated more than once (since we made our `Polygon` class "immutable").

##### Goal 2

Refactor the `Polygons` (sequence) type, into an **iterable**. Make sure also that the elements in the iterator are computed lazily - i.e. you can no longer use a list as an underlying storage mechanism for your polygons.

You'll need to implement both an iterable, and an iterator.




In [1]:
from numbers import Real #for circumradius
import math

In [81]:
import unittest
# Code needed to run unittest in Jupyter
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)

In [176]:
class Polygon:
    """Immutable version of the Polygon class with lazy property evaluation"""
    def __init__(self, num_edges, circumradius):
        if isinstance(num_edges,int) and num_edges > 2:
            self._num_edges = num_edges
        else: #combining ValueError and TypeError message, but should be ok in setting one attribute
            raise ValueError("Number of edges must be an integer greater than 2")
        #self._num_edges = num_edges
        self._num_vertices = num_edges #num vertices is same as num edges, will only set if edge num is valid
        if isinstance(circumradius,Real) and circumradius > 0:
            self._circumradius = circumradius
        else: 
            raise ValueError("Circumradius must be a positive real number")
        #self._circumradius = circumradius
        #Add attributes below for lazy evaluation
        self._interior_angle = None
        self._edge_length = None
        self._apothem = None
        self._area = None
        self._perimeter = None

    @property
    def num_edges(self):
        return self._num_edges
    

    @property
    def num_vertices(self):
        return self._num_vertices
    
    @property
    def circumradius(self):
        return self._circumradius
    
    @property
    def interior_angle(self):
        if self._interior_angle is None:
            self._interior_angle = ((self.num_edges-2)*180)/self.num_edges
        return self._interior_angle 

    @property
    def edge_length(self):
        if self._edge_length is None:
            self._edge_length = 2*self.circumradius*math.sin(math.pi/self.num_edges)
        return self._edge_length

    @property
    def apothem(self):
        if self._apothem is None:
            self._apothem = self.circumradius*math.cos(math.pi/self.num_edges)
        return self._apothem

    @property
    def area(self):
        if self._area is None:
            self._area = self.num_edges*self.edge_length*self.apothem/2
        return self._area

    @property
    def perimeter(self):
        if self._perimeter is None:
            self._perimeter =  self.num_edges*self.apothem
        return self._perimeter

    def __eq__(self,other):
        """Support Polygon equality based on # edges and circumradius"""
        if isinstance(other,Polygon):
            return self.num_edges == other.num_edges  and self.circumradius == other.circumradius
        else:
            raise TypeError("Comparison not supported between a Polygon and non-Polygon")

    def __lt__(self,other):
        """Support Polygon comparison based on number of edges only"""
        if isinstance(other,Polygon):
            return self.num_edges < other.num_edges 
        else:
            raise TypeError("Comparison not supported between a Polygon and non-Polygon")

    def __repr__(self): 
        return f"Object {Polygon.__name__} with id {id(self)}, circumradius {self.circumradius} and {self.num_edges} edges"



    def __str__(self):
        return f"Polygon with {self.num_edges} edges and circumradius of {self.circumradius}"


### Polygon unit tests

In [118]:
class TestPolygon(unittest.TestCase):
    """Tester for the Polygon class"""
    def setUp(self):
        self._num_edges = 5
        self._num_vertices = self._num_edges #num vertices is same as num edges
        self._circumradius = 7.0
        self._poly = Polygon(self._num_edges, self._circumradius)
    
    def test_ok(self):
        self.assertEqual(self._poly.num_edges,self._num_edges)
        self.assertEqual(self._poly.circumradius,self._circumradius)
        self.assertEqual(self._poly.num_vertices,self._num_vertices)
        self.assertEqual(self._num_edges,self._num_vertices)
        self.assertAlmostEqual(self._poly.interior_angle, 108.0)
        self.assertAlmostEqual(self._poly.edge_length, 8.2289935)
        self.assertAlmostEqual(self._poly.apothem, 5.663118960)
        self.assertAlmostEqual(self._poly.area, 116.50442324615)
        self.assertAlmostEqual(self._poly.perimeter, 28.315594803)

    def test_invalid_circumradii(self):
        test_circumradii = (-1, 0,'ten')
        for i, test_c in enumerate(test_circumradii):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    Polygon(self._num_edges, circumradius= test_c)
                    
    def test_invalid_edges(self):
        test_edges = (-1, 0,'ten', 10.0)
        for i, test_e in enumerate(test_edges):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    Polygon(num_edges=test_e, circumradius=self._circumradius)

    def test_equal_ok(self):
        self.assertEqual(self._poly,Polygon(5,7))
        self.assertNotEqual(self._poly,Polygon(5,7.6))
        self.assertNotEqual(self._poly,Polygon(3,7))

    def test_equal_invalid_type(self):
        with self.assertRaises(TypeError):
            self.assertEqual(3.14159,Polygon(5,7))


    def test_lt_ok(self):
        self.assertLess(Polygon(3,7),self._poly)
        self.assertGreater(self._poly,Polygon(3,7))

    def test_lt_invalid_type(self):
        with self.assertRaises(TypeError):
            self.assertLess(3.14159,Polygon(5,7))


    def test_repr_ok(self):
        poly_for_test = Polygon(self._num_edges, self._circumradius)
        self.assertEqual(repr(poly_for_test),
        (f"Object {Polygon.__name__} with id {id(poly_for_test)}, "
         +f"circumradius {self._circumradius} and {self._num_edges} edges"))
  
            
    def test_str_ok(self):
            poly_for_test = Polygon(self._num_edges, self._circumradius)
            self.assertEqual(str(poly_for_test),
            f"Polygon with {self._num_edges} edges and circumradius of {self._circumradius}")

run_tests(TestPolygon)

test_equal_invalid_type (__main__.TestPolygon) ... ok
test_equal_ok (__main__.TestPolygon) ... ok
test_invalid_circumradii (__main__.TestPolygon) ... ok
test_invalid_edges (__main__.TestPolygon) ... ok
test_lt_invalid_type (__main__.TestPolygon) ... ok
test_lt_ok (__main__.TestPolygon) ... ok
test_ok (__main__.TestPolygon) ... ok
test_repr_ok (__main__.TestPolygon) ... ok
test_str_ok (__main__.TestPolygon) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.081s

OK


In [152]:
class Polygons:
    """Polygons iterable class"""
    def __init__(self, max_vertices, circumradius):

        if isinstance(max_vertices,int) and max_vertices > 2:
            self._max_vertices = max_vertices #consider doing this with a property
        else:
            raise IndexError("Index must be an integer greater than 2")

        if isinstance(circumradius,Real) and circumradius > 0:
            self._circumradius = circumradius
        else: 
            raise ValueError("Circumradius must be a positive real number")
        
        self._max_efficiency_polygon = None

    def __len__(self):
        return self._max_vertices - 2

    def __iter__(self):
        return PolygonsIterator(self._max_vertices,self._circumradius)

    def __str__(self):
        return f"Polygons iterable with minimum of 3 edges, maximum of {self._max_vertices} edges, and circumradius {self._circumradius}"
    def __repr__(self):
        return (f"Polygons iterable with minimum of 3 edges, maximum of {self._max_vertices} edges, and circumradius {self._circumradius}"+
            f"\nStored at address {id(self)}")
    @property
    def max_efficiency_polygon(self):
        """Set and return the maximum efficiency polygon with the highest area:perimeter ratio"""
        #Inspired by Fred Baptiste's approach, see my approach below
        sorted_polys = sorted(
            PolygonsIterator(self._max_vertices,self._circumradius),
            key = lambda  poly:poly.area/poly.perimeter,
            reverse=True)
        self._max_efficiency_polygon = sorted_polys[0]
        return self._max_efficiency_polygon 
    
    
    # @property
    # def max_efficiency_polygon(self):
    #     """Set and return the maximum efficiency polygon with the highest area:perimeter ratio"""
    #     #my approach
    #     max_eff,idx = Polygon(3,self._circumradius).area/Polygon(3,self._circumradius).perimeter,3
    #     for i in range(4,self._max_vertices+1):
    #         current_eff = Polygon(i,self._circumradius).area/Polygon(3,self._circumradius).perimeter
    #         if  current_eff > max_eff:
    #             max_eff, idx = current_eff, i
    #     self._max_efficiency_polygon = Polygon(idx,self._circumradius)
    #     return self._max_efficiency_polygon 


class PolygonsIterator:
    """PolygonsIterator supports fully-featured slicing and indexing (positive indices, negative indices, slicing, and extended slicing)
    PolygonsIterator supports the `length()` function
    PolygonsIterator provides the polygon with the highest `area:perimeter` ratio"""
    def __init__(self, max_vertices, circumradius):
        
        if isinstance(max_vertices,int) and max_vertices < 3:
            raise IndexError("Maximum number of vertices must be at least 3")
        if isinstance(max_vertices,int):
            self._max_vertices = max_vertices #consider doing this with a property
        else:
            raise IndexError("Index must be an integer")

        if isinstance(circumradius,Real) and circumradius > 0:
            self._circumradius = circumradius
        else: 
            raise ValueError("Circumradius must be a positive real number")
        
        self.i = 3


    def __iter__(self):
        return self

    def __next__(self):
        if self.i > self._max_vertices:
            raise StopIteration
        else:
            poly = Polygon(self.i,self._circumradius)
            self.i += 1
            return poly

    def __str__(self):
        return f"PolygonsIterator type of up to {self._max_vertices} vertices and with circumradius of {self._circumradius} "
    def __repr__(self):
        return (f"PolygonsIterator type of up to {self._max_vertices} vertices and with circumradius of {self._circumradius} "+
                f"{PolygonsIterator.__name__} with id {id(self)}")
    
    

### Polygons unit tests

In [148]:
class TestPolygons(unittest.TestCase):
    """Tester for the Polygon class"""
    def setUp(self):
        self._max_vertices = 10
        self._circumradius = 7.0
        self._polys = Polygons(self._max_vertices, self._circumradius)
    
    def test_ok(self):
        self.assertEqual(self._polys._max_vertices,self._max_vertices)
        self.assertEqual(self._polys._circumradius,self._circumradius)


    def test_invalid_circumradii(self):
        test_circumradii = (-1, 0,'ten')
        for i, test_c in enumerate(test_circumradii):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    Polygons(self._max_vertices, circumradius= test_c)
                    
    def test_invalid_vertices(self):
        test_vertices = (-1, 0,'ten', 10.0)
        for i, test_v in enumerate(test_vertices):
            with self.subTest(test_number=i):
                with self.assertRaises(IndexError):
                    Polygons(max_vertices=test_v, circumradius=self._circumradius)

    def test_iter(self):
        self.assertIsInstance(iter(self._polys),PolygonsIterator)

    def test_len(self):
        self.assertEqual(len(self._polys),self._max_vertices-2)

    def test_max_efficiency(self):
        self.assertEqual(self._polys.max_efficiency_polygon, Polygon(3,7.0))
  

    def test_repr_ok(self):
        polys_for_test = Polygons(self._max_vertices, self._circumradius)
        self.assertEqual(repr(polys_for_test),
        (f"Polygons iterable with minimum of 3 edges, maximum of {polys_for_test._max_vertices} edges,"
         +f" and circumradius {polys_for_test._circumradius}"
         +f"\nStored at address {id(polys_for_test)}"))

            
    def test_str_ok(self):
            polys_for_test = Polygons(self._max_vertices, self._circumradius)
            self.assertEqual(str(polys_for_test),
            f"Polygons iterable with minimum of 3 edges, maximum of {self._max_vertices}"
            +f" edges, and circumradius {self._circumradius}")
run_tests(TestPolygons)

test_invalid_circumradii (__main__.TestPolygons) ... ok
test_invalid_vertices (__main__.TestPolygons) ... ok
test_iter (__main__.TestPolygons) ... ok
test_len (__main__.TestPolygons) ... ok
test_max_efficiency (__main__.TestPolygons) ... ok
test_ok (__main__.TestPolygons) ... ok
test_repr_ok (__main__.TestPolygons) ... ok
test_str_ok (__main__.TestPolygons) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.018s

OK


### PolygonsIterator unit tests

In [174]:
class TestPolygonsIterator(unittest.TestCase):
    def setUp(self):
        self._max_vertices = 10
        self._circumradius = 7.0
        self.poly_iter = PolygonsIterator(self._max_vertices,self._circumradius)
        self.poly_iter.i = 3
   

    def test_invalid_circumradii(self):
        test_circumradii = (-1, 0,'ten')
        for i, test_c in enumerate(test_circumradii):
            with self.subTest(test_number=i):
                with self.assertRaises(ValueError):
                    PolygonsIterator(self._max_vertices, circumradius= test_c)
                    
    def test_invalid_max_idx(self):
        test_vertices = (-1,2,0)
        for i, test_v in enumerate(test_vertices):
            with self.subTest(test_number=i):
                with self.assertRaises(IndexError):
                    PolygonsIterator(max_vertices=test_v, circumradius=self._circumradius)

    def test_invalid_vertices(self):
        test_vertices = ('ten', 10.0)
        for i, test_v in enumerate(test_vertices):
            with self.subTest(test_number=i):
                with self.assertRaises(IndexError):
                    PolygonsIterator(max_vertices=test_v, circumradius=self._circumradius)

    def test_iter(self):
        self.assertIsInstance(iter(PolygonsIterator(self._max_vertices,self._circumradius)),PolygonsIterator)

    def test_next_raises_stop_iter(self):
        test_is = (11, 20)
        for i, test_i in enumerate(test_is):
            with self.assertRaises(StopIteration):
                self.poly_iter.i = test_i
                next(self.poly_iter)

    def test_next_increments(self):
        cur = self.poly_iter.i
        next(self.poly_iter)
        self.assertEqual(self.poly_iter.i,cur+1)
  

    def test_repr_ok(self):
        poly_iter_for_test = PolygonsIterator(self._max_vertices, self._circumradius)
        self.assertEqual(repr(poly_iter_for_test),
        (f"PolygonsIterator type of up to {poly_iter_for_test._max_vertices} vertices and with circumradius of {poly_iter_for_test._circumradius} "
         +f"{PolygonsIterator.__name__} with id {id(poly_iter_for_test)}"
        )
        )


    def test_str_ok(self):
            poly_iter_for_test  = PolygonsIterator(self._max_vertices, self._circumradius)
            self.assertEqual(str(poly_iter_for_test),
            f"PolygonsIterator type of up to {poly_iter_for_test._max_vertices} vertices and with circumradius of {poly_iter_for_test._circumradius} "
            )

run_tests(TestPolygonsIterator)

test_invalid_circumradii (__main__.TestPolygonsIterator) ... ok
test_invalid_max_idx (__main__.TestPolygonsIterator) ... ok
test_invalid_vertices (__main__.TestPolygonsIterator) ... ok
test_iter (__main__.TestPolygonsIterator) ... ok
test_next_increments (__main__.TestPolygonsIterator) ... ok
test_next_raises_stop_iter (__main__.TestPolygonsIterator) ... ok
test_repr_ok (__main__.TestPolygonsIterator) ... ok
test_str_ok (__main__.TestPolygonsIterator) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.041s

OK
